# Práctica 3: Representación del conocimiento


El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 08__

__Nombres de los estudiantes: Lubin Ye y Ziteng Huang__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural).

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.

Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [2]:
## Lo primero sería instalar en tu entorno de Python wdsparql
## Si no lo tienes instalado, puedes hacerlo desde el notebook
import sys

!{sys.executable} -m pip install wdsparql

# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [2]:
%%wdsparql
SELECT ?item ?itemLabel
WHERE
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q3569037,William Windsor
1,http://www.wikidata.org/entity/Q13050139,Q13050139
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I
7,http://www.wikidata.org/entity/Q117303721,Q117303721
8,http://www.wikidata.org/entity/Q117303807,Q117303807
9,http://www.wikidata.org/entity/Q117303856,Q117303856


### Consulta 1

[Stephen King (Q39829)](https://www.wikidata.org/wiki/Q39829) es un escritor estadounidense de novelas de terror, ficción sobrenatural, misterio, ciencia ficción y literatura fantástica.

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

In [51]:
%%wdsparql
SELECT ?birthDate ?birthPlaceLabel ?countryLabel
WHERE
{
  wd:Q39829 wdt:P569 ?birthDate;  # date of birth
            wdt:P19 ?birthPlace;  # place of birth
            wdt:P27 ?country.     # country of birthplace

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}


,birthDate,birthPlaceLabel,countryLabel
0,1947-09-21T00:00:00Z,Portland,United States of America


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

In [52]:
%%wdsparql
SELECT DISTINCT ?occupationLabel
WHERE
{
  wd:Q39829 wdt:P106 ?occupation.  # instance of human
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?occupationLabel


,occupationLabel
0,actor
1,columnist
2,director
3,film director
4,journalist
5,novelist
6,science fiction writer
7,screenwriter
8,teacher
9,television producer


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

In [56]:
%%wdsparql
SELECT DISTINCT ?occupation ?occupationLabel
WHERE {
  wd:Q39829 wdt:P106 ?occupation.  # Q8496 es el identificador de Stephen King
  ?occupation wdt:P279* wd:Q36180.  # Filtrar subtipos de Escritor
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}



,occupation,occupationLabel
0,http://www.wikidata.org/entity/Q1086863,columnist
1,http://www.wikidata.org/entity/Q6625963,novelist
2,http://www.wikidata.org/entity/Q18844224,science fiction writer
3,http://www.wikidata.org/entity/Q36180,writer
4,http://www.wikidata.org/entity/Q28389,screenwriter


### Consulta 4

King se ha casado una vez, con Tabitha Spruce, con la que tiene varios hijos. Queremos conocer los nombres de esos hijos y sus fechas de nacimiento (si están disponibles). Los resultados deben aparecer ordenados cronológicamente.

In [22]:
%%wdsparql
SELECT ?namesLabel ?birthDate
WHERE
{
  wd:Q39829 wdt:P26 wd:Q284287;  # spouse
            wdt:P40 ?names.  # child
  OPTIONAL { ?names wdt:P569 ?birthDate}  # birthDate

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY  ?namesLabel

,namesLabel,birthDate
0,Joe Hill,1972-06-04T00:00:00Z
1,Naomi King,None
2,Owen King,1977-02-21T00:00:00Z


### Consulta 5

King ha recibido muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha en la que fue premiado (si está disponible) y la obra premiada (si está disponible). Los resultados se deben mostrar ordenados por fecha.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [12]:
%%wdsparql
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>

SELECT ?awardLabel ?date ?workLabel
WHERE
{
  wd:Q39829 p:P166 ?awardStatement.  # award statement
  ?awardStatement ps:P166 ?award.    # award
  OPTIONAL {?awardStatement  pq:P585 ?date;     # date of award
                  pq:P1686 ?work.}              # work

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?date


,awardLabel,date,workLabel
0,Shirley Jackson Award,None,None
1,Hammett Prize,None,None
2,O. Henry Award,None,None
3,Edgar Awards,None,None
4,National Medal of Arts,None,None
5,World Horror Convention Grand Master Award,None,None
6,Grand Prix de l'Imaginaire,None,None
7,Hugo Award for Best Related Work,1982-01-01T00:00:00Z,Danse Macabre
8,Bram Stoker Award for Novel,1996-01-01T00:00:00Z,The Green Mile
9,Grand Prix de l'Imaginaire,1997-01-01T00:00:00Z,Danse Macabre


### Consulta 6

King es sin duda un autor prolífico. ¿Cuántas obras ha escrito a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

In [19]:
%%wdsparql
SELECT (COUNT(?work) AS ?numWorks)
WHERE
{
  ?work wdt:P50 wd:Q39829.  # author of notable work
  ?work wdt:P31 wd:Q7725634.  # instance of literary work
}


,numWorks
0,226


### Consulta 7

Aunque King es mundialmente conocido por sus obras de ficción ([Q8253](https://www.wikidata.org/wiki/Q8253)), es también autor de algunas obras menos conocidas de no-ficción ([Q213051](https://www.wikidata.org/wiki/Q213051)). Queremos saber cuáles son estas obras de no-ficción (o cualquiera de sus subgéneros) y su tipo. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

In [37]:
%%wdsparql
SELECT ?workLabel ?workTypeLabel
WHERE {
  ?work wdt:P50 wd:Q39829.  # Q39829 is the identifier for Stephen King
  ?work wdt:P136/wdt:P279* wd:Q213051;   # Filter non-fiction genre (P136) works and their subgenres (P279*) from work
        wdt:P31 ?workType.  # work type

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?workLabel


,workLabel,workTypeLabel
0,Danse Macabre,written work
1,Danse Macabre,literary dylogy
2,Guns,literary work
3,Head Down,written work
4,On Writing,written work
5,On Writing,written work
6,On Writing,written work


### Consulta 8

De todas las obras que ha publicado, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [ ]:
%%wdsparql
SELECT ?workLabel ?publicationDate
WHERE
{
  ?work wdt:P50 wd:Q39829.  # author of work
  ?work wdt:P577 ?publicationDate.  # publication date
  
  FILTER(YEAR(?publicationDate) >= 1970 && YEAR(?publicationDate) <= 1980).
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?publicationDate


### Consulta 9

Wikidata contiene información actualizada que en ocasiones incluye futuros lanzamientos u obras planificadas. Vamos a dejar por un momento a King y vamos a considerar la obra de otro famoso autor, George R. R. Martin.

[George R. R. Martin (Q181677)](https://www.wikidata.org/wiki/Q181677) es un escritor de fantasía estadounidense conocido sobre todo por su saga "Canción de hielo y fuego". Martin tiene inconclusa la saga y queremos saber si hay alguna información sobre los libros pendientes. Estos serán sus obras sin fecha de publicación asociada. ¿Aparecen? Si lo hacen, ¿cuántas y cuáles son sus títulos? ¿Hay alguna obra pendiente que no sea de esa saga?

In [25]:
%%wdsparql

# Seleccionamos las obras de George R. R. Martin que no tienen fecha de publicación pertenecientes a la saga

SELECT ?workLabel
WHERE
{
  ?work wdt:P50 wd:Q181677 .  # author
  ?work wdt:P179 wd:Q45875 .  # pertenece a la saga "Cancion de hielo y fuego"

  OPTIONAL { ?work wdt:P577 ?publicationDate. }
  
   FILTER(!BOUND(?publicationDate)).
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,workLabel
0,The Winds of Winter
1,Blood of the Dragon
2,The Rise of the Dragon


In [28]:
%%wdsparql

# Seleccionamos el numero de obras de George R. R. Martin que no tienen fecha de publicación pertenecientes a la saga
SELECT (COUNT(?work) AS ?pendingWorks)
WHERE
{
  ?work wdt:P50 wd:Q181677 .  # author
  ?work wdt:P179 wd:Q45875 .  # pertenece a la saga "Cancion de hielo y fuego"

  OPTIONAL { ?work wdt:P577 ?publicationDate. }
  
   FILTER(!BOUND(?publicationDate)).
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,pendingWorks
0,3


In [41]:
%%wdsparql

# Seleccionamos el numero de obras de George R. R. Martin que no tienen fecha de publicación y no pertenecientes a la saga
SELECT ?workLabel
WHERE
{
  ?work wdt:P50 wd:Q181677 .  # author

  OPTIONAL { ?work wdt:P577 ?publicationDate. }

  FILTER(!BOUND(?publicationDate)).
  
  MINUS {?work wdt:P179 wd:Q45875.}  # pertenece a la saga "Cancion de hielo y fuego"
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}


,workLabel
0,Q3795208
1,Black Trump
2,Card Sharks
3,Deuces Down
4,Marked Cards
5,Dead Man's Hand
6,Songs the Dead Men Sing
7,Q18649517
8,The Road Less Traveled
9,Doorways


### Consulta 10

Volvemos de nuevo a King. De todas las obras que ha publicado (independientemente de su tipo y género), queremos recuperar los nombres y fechas de publicación de aquellas escritas en coautoría, así como los nombres de estos coautores.

In [50]:
%%wdsparql

SELECT ?workLabel ?publicationDate ?coAuthorLabel
WHERE
{
  ?work wdt:P50 wd:Q39829;  # autor principal
        wdt:P577 ?publicationDate;  # fecha de publicación
        wdt:P57 ?coAuthor.  # coautor (P57)
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?publicationDate


,workLabel,publicationDate,coAuthorLabel
0,Storm of the Century,1999-01-01T00:00:00Z,Craig R. Baxley
1,Hearts in Atlantis,1999-09-14T00:00:00Z,Scott Hicks


### Consulta 11

¿Con cuántos coautores diferentes ha trabajado King, independientemente del tipo y género de la obra?

In [51]:
%%wdsparql

SELECT (COUNT(DISTINCT ?coAuthor) AS ?numCoAuthors)
WHERE
{
  ?work wdt:P50 wd:Q39829;  # main author
        wdt:P57 ?coAuthor.  # co-author (P57)
}


,numCoAuthors
0,2


### Consulta 12

¿Qué otros personajes famosos nacieron el mismo año que King y tienen su misma nacionalidad? Para cada uno de ellos muestra su nombre y ocupación. Muestra los resultados ordenados por nombre, fecha de nacimiento y ocupación.

In [42]:
%%wdsparql

SELECT ?nameLabel ?occupationLabel
WHERE
{
  wd:Q39829 wdt:P569 ?dateOfBirth;  # date of birth
            wdt:P27 ?country.  # country of birth
  ?name wdt:P569 ?dateOfBirth;
             wdt:P27 ?country;
             wdt:P106 ?occupation.  # occupation
  
  FILTER(?name != wd:Q39829)  # Exclude Stephen King
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?name ?dateOfBirth ?occupation

,nameLabel,occupationLabel
0,Adam Rita,athlete
1,Marsha Norman,music teacher
2,Marsha Norman,screenwriter
3,Marsha Norman,writer
4,Jim Todd,baseball player
5,Larry Blakeney,baseball player
6,Larry Blakeney,American football player
7,Larry Blakeney,head coach
8,Don Felder,singer
9,Don Felder,mandolinist


### Consulta 13

¿Qué otros personajes famosos incluyen King en su nombre completo (name in native language, P1559)? Para cada uno de ellos muestra su nombre, fecha de nacimiento, ocupación y nacionalidad. Muestra los resultados ordenados por nombre, fecha de nacimiento y ocupación.

In [ ]:
%%wdsparql

SELECT ?nameLabel ?dateOfBirth ?occupationLabel ?nationalityLabel
WHERE
{
  ?name wdt:P1559 ?nameLabel;  # name in native language
        wdt:P569 ?dateOfBirth;  # date of birth
        wdt:P106 ?occupation;  # occupation
        wdt:P27 ?nationality.  # nationality
  
  FILTER(CONTAINS(LCASE(?nameLabel), "king"))  # filter names containing "king"
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?nameLabel ?dateOfBirth ?occupationLabel


### Consulta 14

¿Cuántos asertos hay sobre King en Wikidata? Ten en cuenta que puede aparecer tanto como sujeto como objeto de una tripleta.

In [44]:
%%wdsparql

SELECT (COUNT(*) AS ?numAssertions)
WHERE
{
  {
    ?s ?p wd:Q39829.  # King as subject
  }
  UNION
  {
    wd:Q39829 ?p ?o.  # King as object
  }
}


,numAssertions
0,3084


__Fecha de las consultas: